<a href="https://colab.research.google.com/github/Alinehbg/CRP/blob/Joshua_Trial/Extract%20Names/Extract_Wiki_Data_names_try_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install requests
import requests

####testing requests with get and post

In [2]:
#USING GET REQUEST
test_query = """SELECT DISTINCT ?item ?itemLabel WHERE {
    ?item rdfs:label "Baker"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""

#encoded_query = requests.utils.quote(test_query) I dont think we need this
url = f'https://query.wikidata.org/sparql?query={test_query}&format=json'
response = requests.get(url)

data = response.json()
data

{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2658150'},
    'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Baker'}}]}}

In [3]:
#USING POST REQUEST
data = {'query': test_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()
data

# dont think I need to specify the content-type as application/x-www-form-urlencoded while sending post request.
# headers = {'content-type': 'application/x-www-form-urlencoded'}
# response = requests.post('https://query.wikidata.org/sparql?format=json', data=data,headers=headers)


{'head': {'vars': ['item', 'itemLabel']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q2658150'},
    'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Baker'}}]}}

#### functions 

In [22]:
def get_itemlabel(query): 
  name_list = []
  for x in query["results"]["bindings"]: 
    name_list.append(x["itemLabel"]["value"])
  return name_list

def retry(data): 
  response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  while response.status_code == 429:
    retry_after = response.headers.get("Retry-After")
    if retry_after:
      # If the value of "Retry-After" is a number, wait that many seconds
      if retry_after.isdigit():
        time.sleep(int(retry_after))
        # If the value of "Retry-After" is a date, parse it and wait until that time
      else:
        retry_time = datetime.datetime.strptime(retry_after, "%a, %d %b %Y %H:%M:%S GMT")
        if retry_time > datetime.datetime.now():
          time.sleep((retry_time - datetime.datetime.now()).total_seconds())
    # Try the request again
    response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
  res = response.json()
  return res

def find_fname_id(name): 
  find_fn_id_query_male = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  find_fn_id_query_female = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q12308941.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query_male}
  res = retry(data)
  if res['results']['bindings'] == []: 
    data = {'query': find_fn_id_query_female}
    res = retry(data)
  try:
    id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
    #print(f"Cannot find first name id for {name}")
    id = []
  return id

def find_lname_id(name): 
  find_fn_id_query = f'''SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    ?item rdfs:label "{name}"@en;
          wdt:P31 wd:Q101352.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
  }}
  '''
  data = {'query': find_fn_id_query}
  res = retry(data)
  try:
      id = res['results']['bindings'][0]['item']['value'].split('/')[-1]
  except IndexError:
      #print(f"Cannot find last name id for {name}")  
      id = []  
  return id

def find_similar_from_id(id): 
  #find similar names from the ID just found for the target name
  id = id
  find_similar_query = f"""
  SELECT DISTINCT ?item ?itemLabel WHERE 
  {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    {{
      SELECT DISTINCT ?item WHERE {{
        ?item p:P460 ?statement0.
        ?statement0 (ps:P460/(wdt:P279*)) wd:{id}.
      }}
    }}
  }}
  """
  data = {'query': find_similar_query}
  res = retry(data)
  similar_name_list = get_itemlabel(res)
  return similar_name_list

def similar_full_names(name): 
  f_name, l_name = name.split(' ')

  f_name_id = find_fname_id(f_name)
  l_name_id = find_lname_id(l_name)

  if f_name_id == [] and l_name_id != []: 
    fname_similar = []
    lname_similar = find_similar_from_id(l_name_id)
    fname_list = []
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  elif l_name_id == [] and f_name_id != []: 
    lname_similar = []
    fname_similar = find_similar_from_id(f_name_id)
    lname_list = []
    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
  elif f_name_id == [] and l_name_id == []: 
    fname_list = []
    lname_list = []
  else: 
    fname_similar = find_similar_from_id(f_name_id)
    lname_similar = find_similar_from_id(l_name_id)

    fname_list = [sim_fname + ' ' + l_name for sim_fname in fname_similar]
    lname_list = [f_name + ' ' + sim_lname for sim_lname in lname_similar]
  
  similar_full_names = fname_list + lname_list
  pair = {name: similar_full_names}
  return pair

def Merge(dict1, dict2):
  return(dict2.update(dict1))

#### All the queries

In [4]:
#QUERIES
#find female and male politicians
male_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581097.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#male Q6581097

female_politicians_query = '''
SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  {
    SELECT DISTINCT ?item ?itemLabel WHERE {
      ?item wdt:P106 wd:Q82955; 
            wdt:P21 wd:Q6581072.
    }
    ORDER BY RAND()
  }
  FILTER REGEX(?itemLabel, "^[A-Za-z]+ [A-Za-z]+$")
}
LIMIT 10
'''
#female Q6581072

####requests and the rest

In [12]:
data = {'query': male_politicians_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()

male_politicans = [ x["itemLabel"]["value"] for x in data['results']['bindings']]

In [13]:
data = {'query': female_politicians_query}
response = requests.post('https://query.wikidata.org/sparql?format=json', data=data)
data = response.json()

female_politicans = [ x["itemLabel"]["value"] for x in data['results']['bindings']]

In [14]:
for name in male_politicans: 
  print(similar_full_names(name))

{'Friedrich Raschig': ['Fred Raschig', 'Frederick Raschig', 'Fredrik Raschig', 'Fridolin Raschig', 'Frigyes Raschig', 'Fridericus Raschig', 'Friedrik Raschig', 'Frederic Raschig', 'Federico Raschig', 'Frederik Raschig', 'Friederich Raschig', 'Frédéric Raschig', 'Bedřich Raschig', 'Fryderyk Raschig', 'Frederig Raschig', 'Freerk Raschig', 'Frederiko Raschig', 'Frédérik Raschig', 'Fick Raschig', 'Fryco Raschig', 'Bjedrich Raschig']}
{'Alexander Radwan': ['Aleksandar Radwan', 'Aleksandar Radwan', 'Sandy Radwan', 'Alejandro Radwan', 'Sandro Radwan', 'Olexandr Radwan', 'Alistair Radwan', 'Alex Radwan', 'Sándor Radwan', 'Alexandre Radwan', 'Alessandro Radwan', 'Aleksandr Radwan', 'Skender Radwan', 'Alissandru Radwan', 'Alexandru Radwan', 'Oleksandr Radwan', 'Alexandr Radwan', 'Aleksander Radwan', 'Aleksandrs Radwan', 'Lissander Radwan', 'Iskandar Radwan', 'Xander Radwan', 'Aleksandras Radwan', 'Lisandro Radwan', 'Aleksandr Radwan', 'Aleks Radwan', 'Aleksandro Radwan', 'Aleksandar Radwan', 'Li

In [26]:
for name in female_politicans: 
  print(similar_full_names(name)) 

{'Zahra Mohammadzadeh': []}
{'Yukiko Kada': []}
{'Maud Olofsson': ['Maud Olafsen', 'Maud Ólafsson', 'Maud Olavsen', 'Maud Olovson', 'Maud Olafson', 'Maud Olavson', 'Maud Olavsson', 'Maud Olafsson']}
{'Salome I': ['Salome Li', 'Salome Lee']}
{'Oona King': ['Oona Rey', 'Oona König', 'Oona Re', 'Oona Rei', 'Oona Roi', 'Oona Koning']}
{'Carly Fiorina': []}
{'Xie Jun': ['Xie Jūn', 'Xie Jūn', 'Xie Jùn']}
{'Malalai Joya': []}
{'Margaret Hamilton': []}
{'Anna Harrison': []}


In [30]:
print(similar_full_names("Joshua Baker"))

{'Joshua Baker': ['Joshuah Baker', 'Josh Baker', 'Jozue Baker', 'Josua Baker', 'Joosua Baker', 'Yosua Baker', 'Josuah Baker', 'Giosuè Baker', 'Yehoshua Baker', 'Joshua Bäcker', 'Joshua Bakker', 'Joshua Becker', 'Joshua Boulanger', 'Joshua Boulenger']}
